In [211]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

# Importing the modules themselves for reloading
import TradePosition
import TouchArea
import TradingStrategy
import TouchDetection

# Reloading the modules to apply any changes
import importlib
importlib.reload(TradePosition)
importlib.reload(TouchArea)
importlib.reload(TradingStrategy)
importlib.reload(TouchDetection)

from TradePosition import TradePosition, csv_to_trade_positions, plot_cumulative_pl_and_price
from TouchArea import TouchArea
from TradingStrategy import StrategyParameters, TouchDetectionAreas, TradingStrategy 

from TouchDetection import *

In [212]:
# symbol = "NVDA"
# symbol = "MSFT"
# symbol = "AAPL" #
# symbol = "TJX"
# symbol = "JNJ" #
symbol = "AMZN"
# symbol = "AZN"
# symbol = "TSLA" #
# symbol = "TJX" #


# start_date = "2023-01-01 00:00:00"
# end_date =   "2024-01-01 00:00:00"

# start_date = "2024-01-01 00:00:00"
# # end_date =   "2024-08-17 00:00:00"
# end_date =   "2024-09-27 00:00:00"

start_date = "2017-01-01 00:00:00"
end_date =   "2018-01-01 00:00:00"

start_date = "2024-08-19 00:00:00"
end_date =   "2024-08-21 00:00:00"

# start_date = "2024-09-01 00:00:00"
# end_date =   "2024-09-19 00:00:00"

# start_date = "2016-10-25 00:00:00"
# end_date =   "2016-10-26 00:00:00"

# Usage example:
touch_params = BacktestTouchDetectionParameters(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    atr_period=15,
    level1_period=15,
    multiplier=1.4,
    min_touches=3,
    start_time=None,
    end_time='15:55',
    # end_time='11:20',
    use_median=True,
    touch_area_width_agg=np_median,
    use_saved_bars=True,
    rolling_avg_decay_rate=0.85,
    export_bars_path=f'bars/bars_{symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv',
    export_quotes_path=f'quotes/quotes_{symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
)

touch_detection_areas = calculate_touch_detection_area(touch_params)

# touch_detection_areas = calculate_touch_detection_area(symbol, start_date, end_date, atr_period=15, level1_period=15, multiplier=1.4, min_touches=3, start_time=None, end_time='15:55', \
#     use_median=True, touch_area_width_agg=np_median, use_saved_bars=True, export_bars_path=f'bars_{symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv')


minute_intervals:   1%|          | 8/782 [00:02<04:22,  2.95it/s]


KeyboardInterrupt: 

In [187]:
touch_detection_areas.bars

open    high       low   close  volume  \
symbol timestamp                                                               
AMZN   2024-08-20 04:00:00-04:00  178.2900  178.79  178.2900  178.79  4970.0   
       2024-08-20 04:01:00-04:00  178.6000  178.60  178.6000  178.60   573.0   
       2024-08-20 04:02:00-04:00  178.6900  178.69  178.6900  178.69   721.0   
       2024-08-20 04:03:00-04:00  178.6900  178.70  178.6900  178.70   956.0   
       2024-08-20 04:04:00-04:00  178.6900  178.70  178.6900  178.70     0.0   
...                                    ...     ...       ...     ...     ...   
       2024-08-20 19:55:00-04:00  178.5515  178.61  178.5515  178.57  1314.0   
       2024-08-20 19:56:00-04:00  178.5515  178.61  178.5515  178.57     0.0   
       2024-08-20 19:57:00-04:00  178.5700  178.57  178.5500  178.55  1458.0   
       2024-08-20 19:58:00-04:00  178.5800  178.58  178.5800  178.58   286.0   
       2024-08-20 19:59:00-04:00  178.6300  178.63  178.6300  178.63   712.0   

                                  trade_count        vwap  central_value  \
symbol timestamp                                                           
AMZN   2024-08-20 04:00:00-04:00        103.0  178.469805     178.683268   
       2024-08-20 04:01:00-04:00         18.0  178.643647     178.675446   
       2024-08-20 04:02:00-04:00          9.0  178.691012     178.690912   
       2024-08-20 04:03:00-04:00         23.0  178.702605     178.696480   
       2024-08-20 04:04:00-04:00          0.0         NaN            NaN   
...                                       ...         ...            ...   
       2024-08-20 19:55:00-04:00         12.0  178.559113     178.585694   
       2024-08-20 19:56:00-04:00          0.0         NaN            NaN   
       2024-08-20 19:57:00-04:00         14.0  178.567990     178.584908   
       2024-08-20 19:58:00-04:00         12.0  178.591434     178.592063   
       2024-08-20 19:59:00-04:00         25.0  178.624344     178.604891   

                                  is_res  shares_per_trade  avg_volume  \
symbol timestamp                                                         
AMZN   2024-08-20 04:00:00-04:00    True         48.252427         NaN   
       2024-08-20 04:01:00-04:00   False         31.833333         NaN   
       2024-08-20 04:02:00-04:00   False         80.111111         NaN   
       2024-08-20 04:03:00-04:00    True         41.565217         NaN   
       2024-08-20 04:04:00-04:00   False               NaN         NaN   
...                                  ...               ...         ...   
       2024-08-20 19:55:00-04:00   False        109.500000  490.226738   
       2024-08-20 19:56:00-04:00   False               NaN  416.692727   
       2024-08-20 19:57:00-04:00   False        104.142857  588.423421   
       2024-08-20 19:58:00-04:00   False         23.833333  541.351389   
       2024-08-20 19:59:00-04:00    True         28.480000  577.171081   

                                  avg_trade_count  log_return  volatility  
symbol timestamp                                                           
AMZN   2024-08-20 04:00:00-04:00              NaN         NaN    0.000000  
       2024-08-20 04:01:00-04:00              NaN   -0.001063    0.000000  
       2024-08-20 04:02:00-04:00              NaN    0.000504    0.000000  
       2024-08-20 04:03:00-04:00              NaN    0.000056    0.000000  
       2024-08-20 04:04:00-04:00              NaN    0.000000    0.000000  
...                                           ...         ...         ...  
       2024-08-20 19:55:00-04:00         9.128298   -0.000120    0.000229  
       2024-08-20 19:56:00-04:00         7.759053    0.000000    0.000229  
       2024-08-20 19:57:00-04:00         8.738267   -0.000112    0.000201  
       2024-08-20 19:58:00-04:00         9.256242    0.000168    0.000183  
       2024-08-20 19:59:00-04:00        11.976738    0.000280    0.000197  

[960 rows x 14 columns]

In [188]:
touch_detection_areas.bars.dtypes

open                float64
high                float64
low                 float64
close               float64
volume              float64
trade_count         float64
vwap                float64
central_value       float64
is_res                 bool
shares_per_trade    float64
avg_volume          float64
avg_trade_count     float64
log_return          float64
volatility          float64
dtype: object

In [189]:
touch_detection_areas.quotes

bid_price  ask_price  bid_size  \
symbol timestamp                                                          
AMZN   2024-08-20 09:29:59.996349-04:00     177.90     177.92       3.0   
       2024-08-20 09:30:00.001404-04:00     177.90     177.92       3.0   
       2024-08-20 09:30:00.002006-04:00     177.85     177.92       1.0   
       2024-08-20 09:30:00.002473-04:00     177.85     177.91       1.0   
       2024-08-20 09:30:00.002898-04:00     177.85     177.92       1.0   
...                                            ...        ...       ...   
       2024-08-20 16:00:00.278185-04:00     178.86     178.93       2.0   
       2024-08-20 16:00:00.339328-04:00     178.86     178.93       2.0   
       2024-08-20 16:00:00.339329-04:00     178.86     178.94       2.0   
       2024-08-20 16:00:00.405080-04:00     178.86     178.94       2.0   
       2024-08-20 16:00:00.405083-04:00     178.86     178.94       2.0   

                                         ask_size bid_exchange ask_exchange  \
symbol timestamp                                                              
AMZN   2024-08-20 09:29:59.996349-04:00       1.0            Q            Q   
       2024-08-20 09:30:00.001404-04:00       4.0            Q            P   
       2024-08-20 09:30:00.002006-04:00       4.0            Q            P   
       2024-08-20 09:30:00.002473-04:00       1.0            Q            Q   
       2024-08-20 09:30:00.002898-04:00       4.0            Q            P   
...                                           ...          ...          ...   
       2024-08-20 16:00:00.278185-04:00      64.0            Q            Q   
       2024-08-20 16:00:00.339328-04:00      30.0            Q            Q   
       2024-08-20 16:00:00.339329-04:00      36.0            Q            Q   
       2024-08-20 16:00:00.405080-04:00      35.0            Q            Q   
       2024-08-20 16:00:00.405083-04:00       5.0            Q            Q   

                                        conditions tape  
symbol timestamp                                         
AMZN   2024-08-20 09:29:59.996349-04:00        [R]    C  
       2024-08-20 09:30:00.001404-04:00        [R]    C  
       2024-08-20 09:30:00.002006-04:00        [R]    C  
       2024-08-20 09:30:00.002473-04:00        [R]    C  
       2024-08-20 09:30:00.002898-04:00        [R]    C  
...                                            ...  ...  
       2024-08-20 16:00:00.278185-04:00        [R]    C  
       2024-08-20 16:00:00.339328-04:00        [R]    C  
       2024-08-20 16:00:00.339329-04:00        [R]    C  
       2024-08-20 16:00:00.405080-04:00        [R]    C  
       2024-08-20 16:00:00.405083-04:00        [R]    C  

[8644 rows x 8 columns]

In [190]:
touch_detection_areas.quotes.dtypes

bid_price       float64
ask_price       float64
bid_size        float64
ask_size        float64
bid_exchange     object
ask_exchange     object
conditions       object
tape             object
dtype: object

In [ ]:
print(touch_detection_areas.bars[touch_detection_areas.mask].columns.to_list())

In [28]:
# touch_detection_areas.bars[touch_detection_areas.mask].reset_index().to_csv('asdf2.csv',index=False)
# touch_detection_areas.bars.reset_index().to_csv('asdfasdf.csv',index=False)

In [29]:
# a = touch_detection_areas.bars[touch_detection_areas.mask]['trade_count'].astype(float).describe()
# b = touch_detection_areas.bars[touch_detection_areas.mask]['volume'].astype(float).describe()
# c = touch_detection_areas.bars[touch_detection_areas.mask]['shares_per_trade'].astype(float).describe()
# pd.concat([a,b,c],axis=1)

In [30]:
# zoom_start_date = start_date
# zoom_end_date = end_date
# zoom_start_date = "2024-08-20 10:30:00"
# zoom_end_date =   "2024-08-20 11:30:00"
# plot_touch_detection_areas(touch_detection_areas)
# plot_touch_detection_areas(touch_detection_areas, zoom_start_date, zoom_end_date)

In [31]:
len(touch_detection_areas.long_touch_area) + len(touch_detection_areas.short_touch_area)

419

In [32]:
params = StrategyParameters(
    initial_investment=10_000,
    # initial_investment=10103.889074410155,
    do_longs=True,
    do_shorts=True,
    sim_longs=True,
    sim_shorts=True,
    
    use_margin=True,
    
    times_buying_power=1,
    
    soft_start_time = None, 
    soft_end_time = '15:50',
    
    # slippage_factor=0
    # slippage_factor=0.2
    # slippage_factor=0.001
)

strategy = TradingStrategy(touch_detection_areas, params, export_trades_path='trades_output.csv')
results = strategy.run_backtest()

AMZN is marginable.
AMZN is shortable and ETB.


run_backtest:   2%|▏         | 17/771 [00:00<00:13, 55.67it/s]

['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap', 'central_value', 'is_res', 'shares_per_trade', 'avg_volume', 'avg_trade_count', 'log_return', 'volatility']
[np.float64(177.11), np.float64(177.2224), np.float64(177.018), np.float64(177.2224), np.float64(72434.0), np.float64(1097.0), np.float64(177.097763), np.float64(177.29174207857977), np.False_, np.float64(66.02917046490428), np.float64(106376.01421331087), np.float64(1399.7235429965563), np.float64(0.0006344325484776549), np.float64(0.0009153369853577669)]
177.11 177.2224


NameError: name 'volatility' is not defined

In [9]:
asdf

NameError: name 'asdf' is not defined

In [10]:
# symbols = ['AAPL','INTC','JNJ','TSLA','MSFT','AMZN','ABBV']
# symbols = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'AMZN', 'JPM', 'GOOGL', 'META', 'XOM', 'CVX', 'UNH', 'PFE', 'KO', 'PG', 'BA', 'CAT']
# symbols = ['NVDA', 'JPM', 'GOOGL', 'META', 'XOM', 'CVX', 'UNH', 'PFE', 'KO', 'PG', 'BA', 'CAT']
symbols = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'BAC', 'META', 'T', 'VZ', 'XOM', 'CVX', 'JNJ', 'PFE', 'KO'] # test 7. generally high-liquidity from 2016 to present
# symbols = ['AAPL', 'MSFT', 'NVDA', 'INTC', 'AMZN', 'TSLA', 'DIS', 'JPM', 'BAC', 'GOOGL', 'META', 'XOM', 'CVX', 'JNJ', 'PFE', 'KO'] # test3,7. generally high-liquidity from 2016 to 2019

In [23]:
trades_folder = 'test7/trades'
graphs_folder = 'test7/graphs'
xbp_folder = 'test7/xbp'

from IPython.utils import io
for s in symbols:
    touch_params.symbol = s
    touch_params.export_bars_path=f'bars/bars_{touch_params.symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    
    # print(touch_params.symbol)
    # print(touch_params.export_bars_path)
    results_list = []
    
    # print(touch_params)
    touch_detection_areas = calculate_touch_detection_area(touch_params)
    # continue
    for i in tqdm(list(np.arange(0.5, 4.01, 0.5))):
        
        params.times_buying_power = i
        with io.capture_output() as captured:
            strategy = TradingStrategy(touch_detection_areas, params)
            
            if i == 1:
                strategy.export_trades_path=f'{trades_folder}/trades_{touch_params.symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
                strategy.export_graph_path=f'{graphs_folder}/{start_date.split()[0]}_{end_date.split()[0]}/graph_{touch_params.symbol}_{start_date.split()[0]}_{end_date.split()[0]}.png'
            
            balance, longs_executed, shorts_executed, balance_change, mean_plpc, win_mean_plpc, lose_mean_plpc, winrate, total_costs, \
                avg_sub_pos, avg_transact, count_entry_adjust, count_entry_skip, count_exit_adjust, count_exit_skip, key_stats = \
                strategy.run_backtest()

        trades_executed = longs_executed + shorts_executed
        newrow = {
            'xBP': i,
            'net%': balance_change,
            'balance': balance,
            'cnt': trades_executed,
            # 'AvgPL%': mean_plpc,
            # 'winAvgPL%': win_mean_plpc,
            # 'loseAvgPL%': lose_mean_plpc,
            'WR%': winrate,
            'TotalCosts': total_costs,
            # 'avg sub pos created': avg_sub_pos,
            # 'avg transactions': avg_transact,
            'entryAdjust': count_entry_adjust,
            'entrySkip': count_entry_skip,
            'exitAdjust': count_exit_adjust,
            'exitSkip': count_exit_skip,
            **key_stats
        }
        results_list.append(newrow)

    results = pd.DataFrame(results_list)
    # Format specific columns if needed
    results['xBP'] = results['xBP'].map('{:.1f}'.format)
    results['balance'] = results['balance'].map('${:.4f}'.format)
    results['TotalCosts'] = results['TotalCosts'].map('${:.4f}'.format)

    # Format all float columns
    float_columns = results.select_dtypes(include=['float64']).columns
    results[float_columns] = results[float_columns].applymap('{:.4f}'.format)

    # Display the results
    # pd.set_option('display.max_columns', None)
    filename = f'{xbp_folder}/xbp_{touch_params.symbol}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    results.to_csv(filename,index=False)
    # results

NameError: name 'symbols' is not defined

In [672]:
# symbols = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'AMZN', 'JPM', 'GOOGL', 'META', 'XOM', 'CVX', 'UNH', 'PFE', 'KO', 'PG', 'BA', 'CAT','INTC','JNJ','ABBV'] # test2

In [ ]:
results_combined = pd.DataFrame()
for s in symbols:
    fname = f'{xbp_folder}/xbp_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    results = pd.read_csv(fname)
    results['symbol'] = s
    first_column = results.pop('symbol') 
    results.insert(0, 'symbol', first_column) 
    results_combined = pd.concat([results_combined, results],ignore_index=True)
results_combined['WinMax'] = results_combined[['LwinMax', 'SwinMax']].max(axis=1)
results_combined['LoseMin'] = results_combined[['LloseMin', 'SloseMin']].max(axis=1)
pd.set_option('display.max_rows', results_combined.shape[0])
pd.set_option('display.max_columns', results_combined.shape[1])
results_combined.loc[results_combined.xBP.isin({1,4})].sort_values('net%',ascending=False)

In [ ]:
for s in symbols:
    fname = f'{xbp_folder}/xbp_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    results = pd.read_csv(fname)
    plt.plot(results.xBP.astype(float).to_list(), results.AllAvg.astype(float).to_list(), label=s)
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.show()

In [ ]:
for s in symbols:
    fname = f'{xbp_folder}/xbp_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    results = pd.read_csv(fname)
    plt.plot(results.xBP.astype(float).to_list(), results['net%'].astype(float).to_list(), label=s)
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.show()

In [ ]:
for s in symbols:
    fname = f'{xbp_folder}/xbp_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    results = pd.read_csv(fname)
    plt.plot(results.xBP.astype(float).to_list(), results['WR%'].astype(float).to_list(), label=s)
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.show()

In [71]:
import zipfile
from zoneinfo import ZoneInfo
ny_tz = ZoneInfo("America/New_York")
symbols = ['AAPL', 'MSFT', 'NVDA', 'INTC', 'AMZN', 'TSLA', 'DIS', 'JPM', 'BAC', 'GOOGL', 'META', 'XOM', 'CVX', 'JNJ', 'PFE', 'KO']
start_date = "2017-01-01 00:00:00"
end_date =   "2018-01-01 00:00:00"

for s in symbols:
    fname = f'{trades_folder}/trades_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    trades = csv_to_trade_positions(fname)
    
    export_bars_path=f'bars/bars_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    zip_file_path = export_bars_path.replace('.csv', '.zip')

    csv_file = zipfile.ZipFile(zip_file_path, 'r').open(os.path.basename(export_bars_path))
    df = pd.read_csv(csv_file)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_convert(ny_tz)
    df.set_index(['symbol', 'timestamp'], inplace=True)
    fill_missing_data(df)
    print(f'Retrieved bars from {zip_file_path}')
    
    # plot_cumulative_pl_and_price(trades, df, None)
    
    export_graph_path=f'{graphs_folder}/{start_date.split()[0]}_{end_date.split()[0]}/graph_{s}_{start_date.split()[0]}_{end_date.split()[0]}.png'
    plot_cumulative_pl_and_price(trades, df, None, filename=export_graph_path)

Retrieved bars from bars/bars_AAPL_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_MSFT_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_NVDA_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_INTC_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_AMZN_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_TSLA_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_DIS_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_JPM_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_BAC_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_GOOGL_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_META_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_XOM_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_CVX_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_JNJ_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_PFE_2017-01-01_2018-01-01.zip
Retrieved bars from bars/bars_KO_2017-01-01_2018-01-01.zip


In [72]:
import zipfile
from zoneinfo import ZoneInfo
ny_tz = ZoneInfo("America/New_York")
symbols = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'BAC', 'META', 'T', 'VZ', 'XOM', 'CVX', 'JNJ', 'PFE', 'KO'] 
start_date = "2024-01-01 00:00:00"
end_date =   "2024-09-27 00:00:00"

for s in symbols:
    fname = f'{trades_folder}/trades_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    trades = csv_to_trade_positions(fname)
    
    export_bars_path=f'bars/bars_{s}_{start_date.split()[0]}_{end_date.split()[0]}.csv'
    zip_file_path = export_bars_path.replace('.csv', '.zip')

    csv_file = zipfile.ZipFile(zip_file_path, 'r').open(os.path.basename(export_bars_path))
    df = pd.read_csv(csv_file)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_convert(ny_tz)
    df.set_index(['symbol', 'timestamp'], inplace=True)
    fill_missing_data(df)
    print(f'Retrieved bars from {zip_file_path}')
    
    # plot_cumulative_pl_and_price(trades, df, None)
    
    export_graph_path=f'{graphs_folder}/{start_date.split()[0]}_{end_date.split()[0]}/graph_{s}_{start_date.split()[0]}_{end_date.split()[0]}.png'
    plot_cumulative_pl_and_price(trades, df, None, filename=export_graph_path)

Retrieved bars from bars/bars_AAPL_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_MSFT_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_NVDA_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_GOOGL_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_AMZN_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_TSLA_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_JPM_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_BAC_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_META_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_T_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_VZ_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_XOM_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_CVX_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_JNJ_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_PFE_2024-01-01_2024-09-27.zip
Retrieved bars from bars/bars_KO_2024-01-01_2024-09-27.zip
